# Scraping de données STATIQUES

In [2]:
url = "https://www.journaldesfemmes.fr/prenoms/classement/prenoms/les-plus-donnes"

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [4]:
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')

In [5]:
print(soup.prettify()[:1000])

<!--Akamai-ESI:PAGE:setreftime=1624062527/-->
<!DOCTYPE html>
<!--[if lt IE 7 ]> <html lang="fr" class="no-js ie6 lt-ie9 lt-ie8 lt-ie7 oldie"> <![endif]-->
<!--[if IE 7 ]>    <html lang="fr" class="no-js ie7 lt-ie9 lt-ie8 oldie"> <![endif]-->
<!--[if IE 8 ]>    <html lang="fr" class="no-js ie8 lt-ie9 oldie"> <![endif]-->
<!--[if IE 9 ]>    <html lang="fr" class="no-js ie9"> <![endif]-->
<!--[if (gt IE 9)|!(IE)]><!-->
<html class="no-js" lang="fr">
 <!--<![endif]-->
 <head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# journaldesfemmes: http://ogp.me/ns/fb/journaldesfemmes#">
  <script type="text/javascript">
   var _sf_startpt=(new Date()).getTime()
  </script>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="always" name="referrer"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <title>
   Prénoms les plus donnés en France
  </title>
  <script>
   var dataLayer = [{"appConfig":{"subContai

Puis on recherche toutes les valeurs que l'on va rentrer dans notre tableau à commencer par les noms de colonnes :

In [6]:
soup.find_all("th")

[<th class="odGradient">Rang</th>,
 <th class="odGradient">Prénom</th>,
 <th class="odGradient">Naissances en 2019</th>]

On liste les noms de colonnes du tableau :

In [7]:
columns_names = []
for column_name in soup.find_all("th"):
    columns_names.append(column_name.text)

In [8]:
df = pd.DataFrame(columns = columns_names)

In [9]:
df

,Rang,Prénom,Naissances en 2019


## 1 : Récupérer le tableau de la première page

In [10]:
for i in soup.find_all("tr")[1:]:
    
    # On cherche le rang du prénom
    rank = i.find_all("td")[0].text
    # On cherche le prénom ....
    name = i.find_all("td")[1].text
    occurence = i.find_all("td")[2].text
    # La ième occurence
    df = df.append({'Rang': rank, 'Prénom': name, 'Naissances en 2019': occurence}, ignore_index=True)

In [11]:
print(df.head())

  Rang   Prénom Naissances en 2019
0    1  Gabriel   5 010 naissances
1    2      Léo   4 885 naissances
2    3  Raphaël   4 661 naissances
3    4   Arthur   4 005 naissances
4    5    Louis   3 956 naissances


## 2 : Automatiser la récupération du tableau pour chaque page

In [12]:
url = "https://www.journaldesfemmes.fr/prenoms/classement/prenoms/les-plus-donnes?page="
df = pd.DataFrame(columns = columns_names)

In [13]:
numero_page = 1
prenom_0 = "Prénom par défault"
prenom = ""

while prenom_0 != prenom :
    prenom = prenom_0
    print("Nous allons scraper cette page web : " + url + str(numero_page))

    try :
        r = requests.get(url + str(numero_page))
        soup = BeautifulSoup(r.text, 'lxml')
        for i in soup.find_all("tr")[1:]:
            rang = i.find_all("td")[0].text
            prenom = i.find_all("td")[1].text
            occurence = i.find_all("td")[2].text

            df = df.append({'Rang': rang, 'Prénom': prenom, 'Naissances en 2019': occurence}, ignore_index=True)
            
        numero_page += 1
        
    except :
        print("Erreur lors du scraping")
        break

Nous allons scraper cette page web : https://www.journaldesfemmes.fr/prenoms/classement/prenoms/les-plus-donnes?page=1
Nous allons scraper cette page web : https://www.journaldesfemmes.fr/prenoms/classement/prenoms/les-plus-donnes?page=2
Nous allons scraper cette page web : https://www.journaldesfemmes.fr/prenoms/classement/prenoms/les-plus-donnes?page=3
Nous allons scraper cette page web : https://www.journaldesfemmes.fr/prenoms/classement/prenoms/les-plus-donnes?page=4
Nous allons scraper cette page web : https://www.journaldesfemmes.fr/prenoms/classement/prenoms/les-plus-donnes?page=5
Nous allons scraper cette page web : https://www.journaldesfemmes.fr/prenoms/classement/prenoms/les-plus-donnes?page=6
Nous allons scraper cette page web : https://www.journaldesfemmes.fr/prenoms/classement/prenoms/les-plus-donnes?page=7
Nous allons scraper cette page web : https://www.journaldesfemmes.fr/prenoms/classement/prenoms/les-plus-donnes?page=8
Nous allons scraper cette page web : https://www

In [14]:
df

,Rang,Prénom,Naissances en 2019
0,1,Gabriel,5 010 naissances
1,2,Léo,4 885 naissances
2,3,Raphaël,4 661 naissances
3,4,Arthur,4 005 naissances
4,5,Louis,3 956 naissances
...,...,...,...
998,999,François,107 naissances
999,1 000,Aymeric,107 naissances
1000,1 001,Iliana,107 naissances
1001,1 002,Inayah,107 naissances
